In [5]:
import models
from models import combine_bench_prior, create_piBo_benchPrior_relRanks_f24, load_priorband_data,add_rel_ranks,rename_algos,create_incumbent, model, glrt
import pandas as pd
import numpy as np

algorithm="algorithm";benchmark="bench_prior";seed="seed";time="used_fidelity"
algos=["pb_mutation_dynamic_geometric-default-at-target","random_search","hyperband"]
fs=[24]
f_space=np.linspace(1,max(fs),max(fs)).tolist()
benchmarks=["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]
label_dict={'random_search': 'RS', 'hyperband': 'HB', 'pb_mutation_dynamic_geometric-default-at-target': 'PB', 'jahs_cifar10': 'JAHS-C10', 'jahs_colorectal_histology': 'JAHS-CH', 'jahs_fashion_mnist': 'JAHS-FM', 'lcbench-126026': 'LC-126026', 'lcbench-167190': 'LC-167190', 'lcbench-168330': 'LC-168330', 'lcbench-168910': 'LC-168910', 'lcbench-189906': 'LC-189906', 'cifar100_wideresnet_2048': 'PD1-Cifar100', 'imagenet_resnet_512': 'PD1-ImageNet', 'lm1b_transformer_2048': 'PD1-LM1B', 'translatewmt_xformer_64': 'PD1-WMT',"random_search_prior":"RS+Prior","bo":"BO","pibo":"PiBO","bohb":"BOHB","priorband_bo":"PriorBand+BO"}

data_pibo=pd.read_parquet("pibo_benchPrior_relRanks_f24_meta.parquet")
data_pb=pd.read_parquet("priorband_benchPrior_relRanks_f24_meta.parquet")

In [13]:
# User questions: How do PiBO etc. compare on different points in time and prior-quality
loss="value"
f_points=[2,5,10,20]

models={}
for prior in ["at25","bad"]:
    models[prior]={}
    print("\n")
    for f in f_points:
        print(f"Prior: {prior} Fidelity: {f}")
        models[prior][f]=model(formula=f"{loss}~{algorithm}+(1|{benchmark})",data=data_pibo.loc[(data_pibo["used_fidelity"]==f)&(data_pibo[benchmark].str.contains(prior))])
        print(models[prior][f].post_hoc(marginal_vars=algorithm)[0])
        print(models[prior][f].post_hoc(marginal_vars=algorithm)[1],"\n")



Prior: at25 Fidelity: 2
P-values adjusted by tukey method for family of 10 estimates
      algorithm  Estimate  2.5_ci  97.5_ci     SE      DF
1      RS+Prior     2.013  -0.897    4.922  1.337  12.109
2  PriorBand+BO     3.809   0.896    6.723  1.340  12.217
3          PiBO     2.134  -0.779    5.047  1.340  12.217
4            BO     2.274  -0.639    5.188  1.340  12.217
5          BOHB     3.816   0.907    6.726  1.337  12.109
P-values adjusted by tukey method for family of 10 estimates
                       Contrast  Estimate  2.5_ci  97.5_ci     SE        DF  T-stat  P-val  Sig
1   (RS+Prior) - (PriorBand+BO)    -1.797  -2.264   -1.330  0.171  2389.269 -10.502  0.000  ***
2             (RS+Prior) - PiBO    -0.121  -0.588    0.346  0.171  2389.269  -0.709  0.954     
3               (RS+Prior) - BO    -0.262  -0.729    0.205  0.171  2389.269  -1.529  0.543     
4             (RS+Prior) - BOHB    -1.804  -2.202   -1.406  0.146  2388.008 -12.361  0.000  ***
5         (PriorBand+BO)

In [14]:
# User questions: How do RS, HB and PB compare on different points in time and prior-quality
loss="rel_rank"
f_points=[2,5,12]


models={}
for prior in ["at25","bad"]:
    models[prior]={}
    print("\n")
    for f in f_points:
        print(f"Prior: {prior} Fidelity: {f}")
        models[prior][f]=model(formula=f"{loss}~{algorithm}+(1|{benchmark})",data=data_pb.loc[(data_pb["used_fidelity"]==f)&(data_pb[benchmark].str.contains(prior))])
        print(models[prior][f].post_hoc(marginal_vars=algorithm)[0])
        print(models[prior][f].post_hoc(marginal_vars=algorithm)[1],"\n")



Prior: at25 Fidelity: 2
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
  algorithm  Estimate  2.5_ci  97.5_ci     SE      DF
1        PB     1.327   1.275    1.378  0.026  1800.0
2        RS     2.511   2.459    2.562  0.026  1800.0
3        HB     2.162   2.111    2.214  0.026  1800.0
P-values adjusted by tukey method for family of 3 estimates
  Contrast  Estimate  2.5_ci  97.5_ci     SE      DF  T-stat  P-val  Sig
1  PB - RS    -1.184  -1.271   -1.097  0.037  1800.0 -31.966    0.0  ***
2  PB - HB    -0.836  -0.923   -0.749  0.037  1800.0 -22.563    0.0  ***
3  RS - HB     0.348   0.261    0.435  0.037  1800.0   9.403    0.0  *** 

Prior: at25 Fidelity: 5
boundary (singular) fit: see help('isSingular') 

P-values adjusted by tukey method for family of 3 estimates
  algorithm  Estimate  2.5_ci  97.5_ci     SE      DF
1        PB     1.467   1.414    1.520  0.027  1800.0
2        RS     2.576   2.523    2.629  0.027  1800.